Recogida de datos

In [139]:
import sqlite3

def guardar(tabla, url, informacion):
    with sqlite3.connect('datosScraping.db') as conexion:
        cursor = conexion.cursor()
    
        # Verificar si ya existe el registro antes de la inserción
        cursor.execute(f"SELECT COUNT(*) FROM {tabla} WHERE url = ? AND informacion = ?", (url, informacion))
        registroExistente = cursor.fetchone()[0]

        if not registroExistente:
            # Evitar posibles problemas de SQL injection utilizando parámetros de la consulta
            cursor.execute(f"INSERT INTO {tabla} (url, informacion) VALUES (?, ?)", (url, informacion))
            print(f"Guardado en {tabla}: {url}, {informacion}")
            conexion.commit()
        else:
            print(f"Registro existente en {tabla}: {url}, {informacion}")

            
    conexion.close()

In [140]:
#CREAMOS UN ARCHIVO PARA GUARDAR LA URL:
def listaUrls(urlNueva):
    # Verificamos si la URL ya existe en el archivo

    try:
        # Verificamos si el archivo existe
        archivo = open("urls.txt", 'r')
        urlsExistentes = archivo.read().splitlines()
        archivo.close()
    except FileNotFoundError:
        # Si el archivo no existe, lo creamos
        archivo = open("urls.txt", 'w')
        urlsExistentes = []
        archivo.close()


    if urlNueva not in urlsExistentes:
        # Si la URL no existe, añadir al final del archivo
           
        archivo = open("urls.txt", 'a')
        archivo.write(f"{urlNueva}\n")
        print(f"URL añadida: {urlNueva}")
        archivo.close()
        return True
            
    else:
        print(f"La URL ya existe en el archivo: {urlNueva}")
        return False



In [141]:
def raspado(urls, etiquetas):
    import requests
    from bs4 import BeautifulSoup
    from urllib.parse import urlparse

    for url in urls: #Verificamos si hay alguna url inválida
        parsed_url = urlparse(url)
        if not (parsed_url.scheme and parsed_url.netloc):
            print(f"URL inválida: {url}")
            continue
        if listaUrls(url):
            contenido = requests.get(url)
            sopa = BeautifulSoup(contenido.text, 'html.parser')

            for etiqueta in etiquetas:
                elementos = sopa.find_all(etiqueta)
                
                for elemento in elementos:
                    if etiqueta == 'a':
                        # Si la etiqueta es 'a' (enlace), obtener la URL del atributo 'href'
                        informacion = elemento.get('href')
                        if not informacion.startswith(('https:', '//')):
                            continue
                    elif etiqueta == 'img':
                        # Si la etiqueta es 'img' (imagen), obtener la URL del atributo 'src'
                        informacion = elemento.get('src')
                        # Agregar condición para verificar si la URL comienza con 'https' o '//'
                        if not informacion.startswith(('https:', '//')):
                            continue  # Salta a la siguiente iteración si no cumple con la condición
                    else:
                        # En otros casos, obtener el texto del elemento
                        informacion = elemento.text.strip()

                    # Guardar en la base de datos
                    guardar(etiqueta, url, informacion)


In [161]:
listaUrl =  ["https://es.wikipedia.org/wiki/Cocker_spaniel_ingl%C3%A9s",
             "https://es.wikipedia.org/wiki/Collie_de_pelo_largo",
             "https://es.wikipedia.org/wiki/D%C3%A1lmata_(perro)"
             ]
listaEtiquetas = ["h1","p","a","img",]

In [162]:
raspado(listaUrl, listaEtiquetas)

URL añadida: https://es.wikipedia.org/wiki/Cocker_spaniel_ingl%C3%A9s
Guardado en h1: https://es.wikipedia.org/wiki/Cocker_spaniel_ingl%C3%A9s, Cocker spaniel inglés
Guardado en p: https://es.wikipedia.org/wiki/Cocker_spaniel_ingl%C3%A9s, El cocker inglés (en inglés: English Cocker Spaniel) es una raza de perro originaria de Inglaterra que fue desarrollada con la finalidad de perseguir a las gallinetas arreándolas de sus escondites en los arbustos. Su variante estadounidense, el cocker americano, resultó de la adaptación del inglés.
Guardado en p: https://es.wikipedia.org/wiki/Cocker_spaniel_ingl%C3%A9s, Fue reconocido como raza en el siglo XIX, aunque su herencia se remonta a algunos miles de años antes de la de los spaniel originales. El nombre viene del francés antiguo espaigneul, que quiere decir “perro español” en referencia al país de donde venían los perros originales (una raza de perros cazadores y cobradores de caza).
Guardado en p: https://es.wikipedia.org/wiki/Cocker_spaniel

Creación de contenido

In [165]:
#Vamos a realizar una funcion que escoja los párrafos en los que mas veces se repita la palabra indicada
import re

def obtenerTresConMasCoincidencias(lista, palabra):
    # Definir una expresión regular para buscar la palabra clave ignorando mayúsculas y minusculas
    patron = re.compile(re.escape(palabra), flags=re.IGNORECASE)

    # Creamos otra función para contar las veces que aparece las palabra clave en una cadena (ignorando mayúsculas y minusculas)
    def veces(cadena):
        return len(re.findall(patron, cadena.lower()))

    #Ordenamos los str por la cantidad de veces que se repite palabra
    listaOrdenada = sorted(lista, key=lambda x: veces(x), reverse=True)

    # Tomar las tres cadenas con más ocurrencias
    top3 = listaOrdenada[:3]

    return top3

In [166]:
import sqlite3

def generarContenido(palabra):
    # Establecer conexión con la base de datos
    with sqlite3.connect('datosScraping.db') as conexion:
        cursor = conexion.cursor()

        # Buscar información en la base de datos utilizando la cláusula LIKE
        #Buscar un título
        cursor.execute("SELECT * FROM h1 WHERE informacion LIKE ?", ('%' + palabra + '%',))
        h1 = cursor.fetchall()

        #Buscar un parrafos
        cursor.execute("SELECT * FROM p WHERE informacion LIKE ?", ('%' + palabra + '%',))
        p = cursor.fetchall()
        

        #Escoger el mejor contenido
        parrafos = []
        for row in p:
            parrafo = row[2]
            parrafos.append(parrafo)

    top3 = obtenerTresConMasCoincidencias(parrafos,palabra)

    # Generar un archivo HTML con el contenido obtenido
    with open('plantilla.html', 'r', encoding='utf-8') as template_file:
        template = template_file.read()
        
        template = template.replace('<!--Título-->', f'{h1[0][2]}' if h1 else '')
        template = template.replace('<!--Texto_Principal-->', f'{top3[0]}' if top3 else '')
        

    with open('output.html', 'w', encoding='utf-8') as output_file:
        output_file.write(template)




In [167]:
generarContenido('Alaskan Husky')